In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import cv2
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import time

### Resize Train set (128x128) and Convert feather format

In [ ]:
start_time = time.time()
data0 = pd.read_parquet('/bengaliai-cv19/data/train_image_data_0.parquet')
data1 = pd.read_parquet('/bengaliai-cv19/data/train_image_data_1.parquet')
data2 = pd.read_parquet('/bengaliai-cv19/data/train_image_data_2.parquet')
data3 = pd.read_parquet('/bengaliai-cv19/data/train_image_data_3.parquet')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
def Resize(df,size=128):
    resized = {} 
    df = df.set_index('image_id')
    for i in tqdm(range(df.shape[0])):
        
        data = 255 - df.loc[df.index[i]].values.reshape(137,236)
        data = data * (255.0/ data.max())
        
        image = cv2.resize(data,(size,size)).astype(np.uint8)
        
#         img = (data[idx]*(255.0/data[idx].max())).astype(np.uint8)

        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T.reset_index()
    resized.columns = resized.columns.astype(str)
    resized.rename(columns={'index':'image_id'},inplace=True)
    return resized

In [ ]:
data0 = Resize(data0)
data0.to_feather('train_data_0.feather')
del data0
data1 = Resize(data1)
data1.to_feather('train_data_1.feather')
del data1
data2 = Resize(data2)
data2.to_feather('train_data_2.feather')
del data2
data3 = Resize(data3)
data3.to_feather('train_data_3.feather')
del data3

### Resize test set (128x128) and Convert feather format

In [ ]:
start_time = time.time()
data0 = pd.read_parquet('./bengaliai-cv19/data/test_image_data_0.parquet')
data1 = pd.read_parquet('./bengaliai-cv19/data/test_image_data_1.parquet')
data2 = pd.read_parquet('./bengaliai-cv19/data/test_image_data_2.parquet')
data3 = pd.read_parquet('./bengaliai-cv19/data/test_image_data_3.parquet')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
data0 = Resize(data0)
data0.to_feather('test_data_0.feather')
del data0
data1 = Resize(data1)
data1.to_feather('test_data_1.feather')
del data1
data2 = Resize(data2)
data2.to_feather('test_data_2.feather')
del data2
data3 = Resize(data3)
data3.to_feather('test_data_3.feather')
del data3

### Reload trainset and Check the images

In [ ]:
start_time = time.time()
data0 = pd.read_feather('train_data_0.feather')
data1 = pd.read_feather('train_data_1.feather')
data2 = pd.read_feather('train_data_2.feather')
data3 = pd.read_feather('train_data_3.feather')
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
def Grapheme_plot(df):
    df_sample = df.sample(15)
    im_id, img = df_sample.iloc[:,0].values,df_sample.iloc[:,1:].values.astype(np.float)
    
    fig,ax = plt.subplots(3,5,figsize=(20,20))
    for i in range(15):
        j=i%3
        k=i//3
        ax[j,k].imshow(img[i].reshape(128,128), cmap='gray')
        ax[j,k].set_title(im_id[i],fontsize=20)
    plt.tight_layout()
        

In [ ]:
Grapheme_plot(data0)
Grapheme_plot(data1)
Grapheme_plot(data2)
Grapheme_plot(data3)